# simplifying the model to mitigate overfitting

Overfitting occurs when the model is 'too complex' for the amount of training data you have available. The 'excess complexity' of the model - relative to the data - allows the model to 'tweak' its parameter values so that it 'overfits' the patterns in the data and begins to fit the statistical 'error' in the data set.

One way to mitigate model overfitting is to design the model so that it is *just complex enough* to fit the expected *patterns* in the data, removing from the model any 'excess complexity' that could be used to overfit the training data.

In the case of probabilistic modeling, the statistical model is specially crafted from expert knowledge of the particular problem domain. The structure and parameters of the model are specified directly from domain-knowledge, so there is a direct link between the model and the expected *processes* that generated the data. In these cases, it's relatively easy to 'tune' the model to match the expected patterns in the data.

Unfortunately, neural networks are *not* typically built directly from domain-knowledge. Rather, the majority of neural networks are *general purpose* approaches that can be *applied* to a wide variety of different problems in many different domains. The 'general' nature of neural networks can be a benefit in many cases, because you can use the same type of model in many different problem domains. However, it does make it more difficult to reliably *tune* the neural network model to capture the expected complexity of the data, because there is no direct link between the parameters of the neural network and the 'parameters' of the problem domain.

In general, the statistical 'complexity' of a model is related to the number of 'free parameters' in the model (also called "trainable parameters" in AI lingo). The more free parameters you have in your model, the higher the model's 'complexity'. Of course, this is a rough correlation, not absolute. But in general, reducing the number of free parameters in a model will typically reduce its complexity and mitigate its potential to 'overfit' training data.

A large amount of research effort in neural networks is focused on developing new 'types' of neurons that maintain the exceptional epxressive power of the network while simultaneously reducing the number of trainable parameters. Convolution networks, recurrent networks and multi-head attention (aka, "transformers") are all types of neurons that were designed with the explicit goal of achieving high expressivity with low parameter counts.

Without resorting to 'fancy' neuron types, we can control the 'complexity' of our neural network by changing its shape (ie, the depth of the network and the widths of its layers).

Adding neurons to a network increases the network's parameter count, which can be done by increasing either network depth or layer width. Removing neurons from the network - by removing either entire layers or reducing the width of each layer - will reduce the network's parameter count.

Of course, if you remove *too many* neurons from your network, you increase the likelihood that your model will *underfit* the training data, leading to potentially biased and unreliable results. Biases introduced by underfitting can cause very serious problems for statistical analyses.

In this notebook, we'll start with an 'overspecified' neural network that clearly overfits its training data. We'll then systematically reduce the 'complexity' of the model to reduce overfitting.

In the below code cell, we simulate a linear data set of 100 samples with 2 explanatory variables and a single response variable.

We split the data set into 60% training and 40% validation data, and we train a complex neural network for 500 epochs using a batch size of 10, stochastic gradient descent optimization and mean squared error loss.

In [ ]:
import sklearn.datasets
import sklearn.model_selection
import matplotlib.pyplot as plt
import tensorflow as tf

# simulate data
x,y = sklearn.datasets.make_regression(n_samples=100,
                                       n_features=2,
                                       bias=100.0,
                                       noise=10.0,
                                       random_state=201188)
y /= 100.0

# partition into train and validation subsets
train_x, valid_x, train_y, valid_y = sklearn.model_selection.train_test_split(x,
                                                                              y,
                                                                              test_size=40,
                                                                              random_state=221882)

# package training and validation data into tensorflow Dataset objects
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(10)
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y)).batch(10)

# build model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=64, activation=tf.keras.activations.relu, input_shape=[2]))
model.add(tf.keras.layers.Dense(units=64, activation=tf.keras.activations.relu))
model.add(tf.keras.layers.Dense(units=64, activation=tf.keras.activations.relu))
model.add(tf.keras.layers.Dense(units=1))

model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss=tf.keras.losses.MeanSquaredError())
model.summary()

# fit model
model.fit(train_data, epochs=500, validation_data=valid_data)

# predict using fitted model
y_hat = model.predict(x)

# plot data and model fit
fig = plt.figure(figsize=(12,12))
axs = fig.add_subplot(projection='3d')
axs.scatter(train_x[:,0], train_x[:,1], train_y, marker='o', s=64)
axs.scatter(valid_x[:,0], valid_x[:,1], valid_y, marker='x', s=64)
axs.scatter(x[:,0], x[:,1], y_hat, marker='+', s=64)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                192       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 8,577
Trainable params: 8,577
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
6/6 [==============================] - 1s 25ms/step - loss: 1.2636 - val_loss: 1.1375
Epoch 2/500
6/6 [==============================] - 0s 4ms/step - loss: 0.7210 - val_loss: 0.8212
Epoch 3/500
6/6 [==============================] - 0s 4ms

As we've seen before, this model clearly overfits it's training data, as evidenced by the final training "loss" being much less than the model's loss on the validation data ("val_loss").